In [62]:
import pandas as pd
import numpy as np 
from tensorflow.keras.layers import Embedding , Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [63]:
df_train = pd.read_csv('train/train.csv')
df_test = pd.read_csv('test/test.csv')
test_id = df_test['ID'].copy()

In [64]:
df_train.shape

(20972, 9)

In [65]:
df_train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [66]:
max = 0
for i in range(len(df_train['ABSTRACT'])):
    
    if len(df_train['ABSTRACT'][i].split()) > max:
        max = len(df_train['ABSTRACT'][i].split())
        print(len(df_train['ABSTRACT'][i].split()))
print(max)

265
274
279
284
286
292
310
354
419
449
449


In [67]:
df_test.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [68]:
df_test.isna().sum()

ID          0
TITLE       0
ABSTRACT    0
dtype: int64

In [69]:
df_train['data'] = df_train['TITLE'].str.cat(df_train['ABSTRACT'], sep='')
df_test['data'] = df_test['TITLE'].str.cat(df_test['ABSTRACT'], sep='')

In [70]:
x = df_train['data']
x_test = df_test['data']
y= df_train.iloc[:,3:-1]
y.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0


In [71]:
doc_size = 10000

In [72]:
x

0        Reconstructing Subject-Specific Effect Maps  P...
1        Rotation Invariance Neural Network  Rotation i...
2        Spherical polyharmonics and Poisson kernels fo...
3        A finite element approximation for the stochas...
4        Comparative study of Discrete Wavelet Transfor...
                               ...                        
20967    Contemporary machine learning: a guide for pra...
20968    Uniform diamond coatings on WC-Co hard alloy c...
20969    Analysing Soccer Games with Clustering and Con...
20970    On the Efficient Simulation of the Left-Tail o...
20971    Why optional stopping is a problem for Bayesia...
Name: data, Length: 20972, dtype: object

In [73]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
import nltk

In [74]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
corpus =[]
pc = PorterStemmer()
for i in range(len(x)):
    data = re.sub('[^a-zA-Z]', ' ', x[i])
    data = data.lower()
    data = data.split()
    
    data = [pc.stem(word)  for word in data if word not in stopwords.words('English')]
    data = ' '.join(data)
    corpus.append(data)

In [ ]:
oH = [one_hot(word, 10000) for word in corpus]

In [ ]:
oH[0]

In [ ]:
padded_words = pad_sequences(oH, padding='pre', maxlen=500)

In [ ]:
model = Sequential()
model.add(Embedding(10000, 50, input_length=500))
model.add(LSTM(100))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
x_words = np.array(padded_words)

In [ ]:
x_words.shape

In [ ]:
y.shape

In [ ]:
from sklearn.mmodel_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_words, y, test_size=0.2, random_state=3)

In [ ]:
model.fit(x_train, y_train, validation_split=(x_val, y_val), epochs=20)